# Training
Este notebook inspirado en el colab anterior esta hecho para poder realizar entrenamiento simple

In [2]:
# !cd ../data && wget https://descargas.inf.santiago.usm.cl/train/4.tar.gz .

--2024-08-13 08:25:30--  https://descargas.inf.santiago.usm.cl/train/4.tar.gz
Resolviendo descargas.inf.santiago.usm.cl (descargas.inf.santiago.usm.cl)... 200.1.22.243
Conectando con descargas.inf.santiago.usm.cl (descargas.inf.santiago.usm.cl)[200.1.22.243]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 8273601942 (7,7G) [application/octet-stream]
Guardando como: ‘4.tar.gz’

4.tar.gz            100%[===================>]   7,71G  11,1MB/s    en 12m 7s  

2024-08-13 08:37:37 (10,9 MB/s) - ‘4.tar.gz’ guardado [8273601942/8273601942]

--2024-08-13 08:37:37--  http://./
Resolviendo . (.)... falló: No existe ninguna dirección asociada al nombre.
wget: no se pudo resolver la dirección del equipo ‘.’
ACABADO --2024-08-13 08:37:37--
Tiempo total de reloj: 12m 7s
Descargados: 1 ficheros, 7,7G en 12m 7s (10,9 MB/s)


In [3]:
# !tar -xf ../data/4.tar.gz -C ../data

In [4]:
#!pip install scikit-optimize

# Codigo de Optimizacion
Definimos los parametros a modificar para realizar proceso de optimizacion

In [13]:
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Integer, Categorical
from skopt.callbacks import CheckpointSaver

from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_histogram, plot_objective_2D

from skopt.utils import use_named_args

# guardar informacion
import pickle
import os




# Codigo

In [7]:
import matplotlib.pyplot as plt
from grav_lens import load_tf_dataset

import numpy as np
import tensorflow as tf

#data
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
# training and modeling
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D


def plot_images(images, cls_true, cls_pred=None, **kwargs):
    assert len(images) == len(cls_true) == 9
    img_shape = images[0].shape
    # Create figure with 3x3 sub-plots.
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Plot image.
        ax.imshow(images[i].reshape(img_shape), cmap='binary')

        # Show true and predicted classes.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)

        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])

    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()



def log_dir_name(learning_rate, activation, batch_size):
    """Helper function para usar tensorboard"""
    # The dir-name for the TensorBoard log-dir.
    s = "./logs/lr_{0:.0e}_act_{1}_batch_{2}/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    # Insert all the hyper-parameters in the dir-name.
    log_dir = s.format(learning_rate,
                       activation,
                       batch_size)

    return log_dir

In [16]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

def load_dataset(data_index='4', max_files=100, home='..'):
    dataset = load_tf_dataset(data_index=data_index, max_files=max_files, home=home)
    return dataset

def split_dataset(dataset, val_split=0.2, test_split=0.1):
    # Convertir el dataset a una lista para dividirlo
    data_list = list(dataset)
    train_data, temp_data = train_test_split(data_list, test_size=val_split + test_split)
    val_data, test_data = train_test_split(temp_data, test_size=test_split / (val_split + test_split))
    
    # Convertir de nuevo a dataset de TensorFlow
    train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
    val_dataset = tf.data.Dataset.from_tensor_slices(val_data)
    test_dataset = tf.data.Dataset.from_tensor_slices(test_data)
    
    return train_dataset, val_dataset, test_dataset

def prepare_dataset(dataset, batch_size=32, shuffle_buffer=1000):
    dataset = dataset.shuffle(buffer_size=shuffle_buffer)  # Mezclar datos
    dataset = dataset.batch(batch_size)  # Batching
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)  # Prefetch para optimización
    return dataset

def get_datasets(data_index='4', max_files=100, home='..', batch_size=32, val_split=0.2, test_split=0.1):
    # Cargar el dataset
    dataset = load_dataset(data_index, max_files, home)
    
    # Dividir el dataset en entrenamiento, validación y prueba
    train_dataset, val_dataset, test_dataset = split_dataset(dataset, val_split, test_split)
    
    # Preparar cada dataset
    train_dataset = prepare_dataset(train_dataset, batch_size)
    val_dataset = prepare_dataset(val_dataset, batch_size)
    test_dataset = prepare_dataset(test_dataset, batch_size)
    
    return train_dataset, val_dataset, test_dataset

# Ejemplo de uso
train_dataset, val_dataset, test_dataset = get_datasets(data_index='4', max_files=100, home='..')

for X, Y in train_dataset.take(1):  # Mostrar un batch de entrenamiento
    print("Train X:", X.shape)
    print("Train Y:", Y.shape)

for X, Y in val_dataset.take(1):  # Mostrar un batch de validación
    print("Val X:", X.shape)
    print("Val Y:", Y.shape)

for X, Y in test_dataset.take(1):  # Mostrar un batch de prueba
    print("Test X:", X.shape)
    print("Test Y:", Y.shape)


Using data folder: ../data/4


2024-08-13 12:03:11.159648: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Shapes of all inputs must match: values[0].shape = [3,128,128] != values[1].shape = [128,128]


InvalidArgumentError: {{function_node __wrapped__Pack_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [3,128,128] != values[1].shape = [128,128] [Op:Pack] name: 0

In [12]:
import tensorflow as tf

# Definir una pérdida personalizada que combine MAE y KL Divergence
def combined_loss(y_true, y_pred):
    mae_loss = tf.keras.losses.mae(y_true, y_pred)  # Usar la función abreviada mae()
    kl_loss = tf.keras.losses.kld(y_true, y_pred)   # Usar la función abreviada kld()
    return mae_loss + kl_loss  # Puedes ponderar las pérdidas si es necesario


def create_model(learning_rate=1e-4, activation="relu"):
    """
    Una arquitectura fija y jugamos con el learning rate
    Hyper-parameters:
        learning_rate:     Learning-rate for the optimizer.
        activation:        Activation function for hidden layers
    """
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(3, 128, 128)),  # Usar la forma original
        tf.keras.layers.Permute((2, 3, 1)),  # Cambiar a (128, 128, 3) para Conv2D
        tf.keras.layers.Conv2D(32, (5, 5), activation=activation, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, (5, 5), activation=activation, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(128, (5, 5), activation=activation, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(1, (5, 5), activation=activation, padding='same')  # Salida con 1 canal
    ])

    model.compile(optimizer=tf.optimizers.Adam(learning_rate=learning_rate), loss=combined_loss)

    return model



# Crear el modelo
model = create_model()
model.summary()




Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ permute_2 (Permute)             │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 128, 128, 32)   │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 128, 128, 64)   │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 128, 128, 128)  │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 128, 128, 1)    │         3,201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 262,721 (1.00 MB)

 Trainable params: 262,273 (1.00 MB)

 Non-trainable params: 448 (1.75 KB)

In [15]:
# Definiendo la forma de pasar parametros a la funcion de optimizacion
dim_learning_rate = Real(low=1e-6, high=1e-2, prior='log-uniform',
                         name='learning_rate')
#dim_num_dense_layers = Integer(low=1, high=5, name='num_dense_layers')
dim_activation = Categorical(categories=['relu', 'sigmoid', 'tanh'],
                             name='activation')
dim_batch_size = Integer(low=16, high=64, name='batch_size')


dimensions = [dim_learning_rate, dim_activation, dim_batch_size]
dimension_names = [x.name for x in dimensions]
default_parameters = [1e-5, 'relu', 32]

In [ ]:
import time

# Inicializar best_accuracy
best_accuracy = 0
verbose = 1

# Función objetivo
@use_named_args(dimensions=dimensions)
def F_objective(learning_rate, activation, batch_size):
    """
    Una arquitectura fija y jugamos con el learning rate, activacion y batchs
    Hyper-parameters:
        learning_rate:      Ratio de aprendizaje para optimizacodres
        activation:         Acticion capas internas
        batch_size:         Organizacion del batch para adtos
    """
    # Crear el modelo
    model = create_model(learning_rate, activation)
    prepared_dataset = prepare_dataset(dataset, batch_size)

    log_dir = log_dir_name(learning_rate, activation, batch_size)
    tensorboard_callback = TensorBoard(
        log_dir=log_dir,
        histogram_freq=0,
        write_graph=True)


    # Entrenar el modelo con TensorBoard callback
    model.fit(prepared_dataset, epochs=1, verbose=verbose, callbacks=[tensorboard_callback])

    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)


    # Print the classification accuracy.
    print()
    print("Accuracy: {0:.2%}".format(accuracy))
    print()

    # Guardamos la mejor precision
    global best_accuracy

    # En caso de mejora
    if accuracy > best_accuracy:
        # Salvar modelo
        model.save_weights('best_model.weights.h5')
        model_id = save_to_drive('best_model.weights.h5', 'best_model.weights.h5')
        print(f"Model weights saved with ID: {model_id}")

        # Update the classification accuracy.
        best_accuracy = accuracy

    # Borrar el modelo y los hiperparametro de memoria
    del model
    # y reiniciar el grafo computacional
    K.clear_session()
    return -accuracy  # Queremos maximizar la precisión, por lo que minimizamos -precisión

# Comprobar funcionamiento
start_time = time.time()
F_objective(x=default_parameters)
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Execution time: {execution_time_minutes:.2f} minutes")

Codigo de minimizacion
Posiblemente tome bastante tiempo en ejecutarse

In [ ]:
# Crear un callback para guardar checkpoints
checkpoint_saver = CheckpointSaver("checkpoint.pkl", compress=9)

# Ejecutar la optimizacion
start_time = time.time()
res = gp_minimize(func=F_objective,
                    dimensions=dimensions,
                    acq_func='EI', # Expected Improvement.
                    n_calls=100,
                    x0=default_parameters,
                    callback=[checkpoint_saver])
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Execution time: {execution_time_minutes:.2f} minutes")

# Guardar resultados
Finalmente se guarda el resultado del training en un modelo pickle, de manera que pueda ser utilizado despues.

## Mision
Guardar los pesos del modelo para que pueda ser reconstruido en local y hacer inferencia